In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [71]:
# generate Header and payload, both assumed to be random
pl_size = 5#640*480*4
H_size = 2#64
header = np.random.randint(2,size=H_size)
pLoad = np.random.randint(2,size=pl_size)
m = np.append(header, pLoad)
if m[0] == 1:
    m = np.append([1,0,1,0], m)
else:
    m = np.append([0,1,0,1], m)


In [96]:
#BPSK modulator and mixing

minv = ~m%2     #inverted message for 0 in BPSK scheme
int_freq = 2 
minv_up = np.repeat(minv,2*int_freq)
m_up = np.repeat(m,2*int_freq)
t = np.linspace(-1,1,2*int_freq*len(m))
A = 1

phi1 = np.cos(2*np.pi*int_freq*t)
phi0 = -np.cos(2*np.pi*int_freq*t)

X1 = np.multiply(phi1, m_up)
X0 = np.multiply(phi0, minv_up)
X = (X1+X0)*A



In [122]:
#AWGN channel and Hard decoding

#generate variance from Eb/N0 array
EbN0dB = np.linspace(-3,12)
EbN0 = 10 ** (EbN0dB/10)
p = A ** 2
R = 1
var = (p/R)/(2*EbN0)
var_idx = 0
mue = 0
z = np.random.normal(mue,np.sqrt(var[var_idx]),len(X))
Y = z+X
print(z)

[ 0.34934216  1.98234553 -1.17541685 -2.12857152  0.17417485 -0.02475474
 -0.50610002 -1.87997667  0.26206363 -1.8003702   0.72079953  0.2363706
  0.01005094 -0.36542734 -0.95483684 -1.02383786 -0.98410685  0.55573064
  1.52369391 -0.37608051  0.38248988  0.04154008 -0.2489246  -0.9948409
 -1.58671632  0.00581403 -0.84781625 -1.14459461 -0.20792009  0.1079133
  1.09370461 -0.12174992  0.61078779 -1.08361134  1.1420411   1.50279147
  0.58316593 -0.02280359 -3.06017991 -0.70083717 -1.41915699 -0.21399445
 -1.20237145 -0.94822844]
